# Leverage Firestore as a Vector Store (GENAI109)

## Overview
In this lab, you'll build a real-world Generative AI Question-and-Answer (Q&A) solution using a Retrieval-Augmented Generation (RAG) framework. You will use Firestore as a vector database to store document chunks and embeddings generated with Vertex AI. By querying Firestore with user input, you'll retrieve relevant information, and use Gemini 2.0 Flash to generate natural language answers.

Note: To avoid confusion between your professional Google Identity and any other temporary Qwiklabs student accounts, it is strongly recommended that you utilize a new Incognito window for the Google Cloud console and Google Drive tabs you will use in this lab. To do this easily in Chrome, after starting the lab, right-click on the “Open Google Cloud console” button and select “Open link in incognito window”.

## Objectives
In this lab, you learn how to:
 - Load and process a document into semantically meaningful chunks
 - Generate vector embeddings using Vertex AI
 - Store chunks and embeddings in Firestore for vector search
 - Query Firestore using user input and retrieve relevant chunks
 - Use Gemini 2.0 Flash to generate natural language answers in a RAG pipeline

## Task 1. Create a Colab Enterprise Notebook

### Upgrade the Vertex AI SDK & Restart the Kernel

5. Paste the following code into the top cell and run it with Shift + Return. If you don’t already have an active notebook runtime, running a cell in a Colab Enterprise notebook will trigger it to create one for you and connect the notebook to it. When a runtime is allocated for the first time, you may be presented with a pop-up window to authorize the environment to act as your Qwiklabs student account.

In [1]:
!pip install --quiet --upgrade google-cloud-logging google_cloud_firestore google_cloud_aiplatform langchain langchain-google-vertexai langchain_community langchain_experimental pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.6/364.6 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44

7. Restart the runtime: select **Runtime > Restart Session > select Yes**. The runtime will restart, indicated by clearing the green checkmark and the cell run order integer next to the cell you ran above.

### Import packages
8. Once the runtime has restarted, paste the following code into the new cell at the bottom of the notebook. (If no new cell exists, you can create one using the + CODE button). Press Shift + Enter to run the cell and import the necessary libraries for embedding generation, text chunking, and Firestore integration:

In [1]:
import vertexai
import logging
import google.cloud.logging
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel

import pickle
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

9. Paste the following into a new code block & run it to initialize Vertex AI.

In [2]:
PROJECT_ID = "qwiklabs-gcp-01-205ae901edae"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

## Task 2. Load an embedding model, precomputed chunks and embeddings
In this section, you will load the preprocessed content and corresponding text embeddings for the NYC Food Safety Manual. These were previously computed and saved in a JSON file. You'll use these chunks and embeddings as input for Retrieval-Augmented Generation (RAG) in later steps.

1. Create a new code cell and run the following to load text embeddings.

In [3]:
embedding_model = VertexAIEmbeddings(model_name="text-embedding-005")

2. Download the preprocessed chunk and embedding data from the NYC Food Protection Training Manual.

In [4]:
!gcloud storage cp gs://qwiklabs-gcp-01-205ae901edae-bucket/embeddings.json .

Copying gs://qwiklabs-gcp-01-205ae901edae-bucket/embeddings.json to file://./embeddings.json


3. Run the following code in a cell to load all the chunk and embedding data from the JSON file into chunks and embeddings lists.

In [5]:
import json

with open("embeddings.json", "r") as f:
    precomputed = json.load(f)

chunks = []
embeddings = []

for item in precomputed:
    chunks.append(item["content"]) # Extract the text chunk
    embeddings.append(item["embedding"]) # Extract the embedding vector

## Task 3. Query Firestore and Generate Answer
In this task, you will set up a Firestore database to store the processed NYC Food Safety Manual chunks and their embeddings for efficient retrieval. You'll then build a search function to find relevant information based on a user query.

1. In the Google Cloud Console, use the **Navigation menu** or the Search box to navigate to **Firestore**.

2. On the Firestore Dashboard, click **CREATE A FIRESTORE DATABASE**. In the setup wizard, configure the database using the following settings, and then click the Create Database button:

<table>
 <tr><td>Property</td><td>Value</td></tr>
 <tr><td>Database ID</td><td>(default)</td></tr>
 <tr><td>Edition</td><td>Standard Edition</td></tr>
 <tr><td>Configuration options</td><td>Firestore Native</td></tr>
 <tr><td>Location type</td><td>Multi-region as nam5 (United States)</td></tr>
</table>

3. Once the firestore database is created, return to the colab environment and initialize the Firestore client and store each embedding and chunk into a collection named food_safety_chunks. Paste and run the following code in a new notebook cell:

In [6]:
from google.cloud import firestore

# Initialize Firestore client
db = firestore.Client(project=PROJECT_ID)
collection = db.collection("food_safety_chunks")

# Store each embedding and chunk
for i, (embedding, chunk) in enumerate(zip(embeddings, chunks)):
    doc = {
        "embedding": embedding,
        "chunk": chunk
    }
    collection.document(f"chunk_{i}").set(doc)

4. Embed a user query, retrieve the most relevant chunks, and generate an answer.Paste and run the following code in a new notebook cell:

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def query_firestore(query_text):
    model = TextEmbeddingModel.from_pretrained("text-embedding-005")
    query_embedding = model.get_embeddings([query_text])[0].values

    docs = collection.stream()
    chunks_data = [(doc.to_dict()["chunk"], doc.to_dict()["embedding"]) for doc in docs]

    sims = [(chunk, cosine_similarity([query_embedding], [emb])[0][0]) for chunk, emb in chunks_data]
    top_chunks = sorted(sims, key=lambda x: x[1], reverse=True)[:3]

    return "\n".join([chunk for chunk, _ in top_chunks])

5. Now that you can retrieve the most relevant content chunks from Firestore, use those chunks as context to generate an answer using the Gemini model. Paste and run the following code in a new notebook cell:

In [8]:
query = "What should you do if food is left out overnight?"
relevant_text = query_firestore(query)

chat_model = GenerativeModel("gemini-2.0-flash")  # Instantiate GenerativeModel directly
chat = chat_model.start_chat()

response = chat.send_message(
    f"Use the following to answer the question:\n\n{relevant_text}\n\nQuestion: {query}"
)
print(response.text)

Based on the information provided (food safety is crucial and proper handling ensures safety), if food is left out overnight, you should **not** consume it. Discard it to prevent potential foodborne illness.

